# Auditing Data Quality

In [11]:
"""
Your task is to check the "productionStartYear" of the DBPedia autos datafile for valid values.
The following things should be done:
- check if the field "productionStartYear" contains a year
- check if the year is in range 1886-2014
- convert the value of the field to be just a year (not full datetime)
- the rest of the fields and values should stay the same
- if the value of the field is a valid year in the range as described above,
  write that line to the output_good file
- if the value of the field is not a valid year as described above, 
  write that line to the output_bad file
- discard rows (neither write to good nor bad) if the URI is not from dbpedia.org
- you should use the provided way of reading and writing data (DictReader and DictWriter)
  They will take care of dealing with the header.

You can write helper functions for checking the data and writing the files, but we will call only the 
'process_file' with 3 arguments (inputfile, output_good, output_bad).
"""
import csv
import pprint
import re

INPUT_FILE = 'autos.csv'
OUTPUT_GOOD = 'autos-valid.csv'
OUTPUT_BAD = 'FIXME-autos.csv'

def process_file(input_file, output_good, output_bad):

    with open(input_file, "r") as f:
        good = []
        bad = []
        reader = csv.DictReader(f)
        header = reader.fieldnames
        for row in reader:
            if 'dbpedia.org' in row['URI']:
                year_str = row['productionStartYear'] 
                if year_str != 'NULL':
                    year = int(year_str[0:4])
                    row['productionStartYear'] = year
                    if year >= 1886 and year <= 2014:
                        good.append(row)
                    else:
                        bad.append(row)
                else:
                    bad.append(row)
            



    with open(output_good, "w") as g:
        writer = csv.DictWriter(g, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in good:
            writer.writerow(row)
            
    with open(output_bad, "w") as g:
        writer = csv.DictWriter(g, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in bad:
            writer.writerow(row)


def test():

    process_file(INPUT_FILE, OUTPUT_GOOD, OUTPUT_BAD)


test()

## Cities

In [12]:
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up. In the first exercise we want you to audit
the datatypes that can be found in some particular fields in the dataset.
The possible types of values can be:
- NoneType if the value is a string "NULL" or an empty string ""
- list, if the value starts with "{"
- int, if the value can be cast to int
- float, if the value can be cast to float, but CANNOT be cast to int.
   For example, '3.23e+07' should be considered a float because it can be cast
   as float but int('3.23e+07') will throw a ValueError
- 'str', for all other values

The audit_file function should return a dictionary containing fieldnames and a 
SET of the types that can be found in the field. e.g.
{"field1": set([type(float()), type(int()), type(str())]),
 "field2": set([type(str())]),
  ....
}
The type() function returns a type object describing the argument given to the 
function. You can also use examples of objects to create type objects, e.g.
type(1.1) for a float: see the test function below for examples.

Note that the first three rows (after the header row) in the cities.csv file
are not actual data points. The contents of these rows should note be included
when processing data types. Be sure to include functionality in your code to
skip over or detect these rows.
"""
import codecs
import csv
import json
import pprint

CITIES = 'cities.csv'

FIELDS = ["name", "timeZone_label", "utcOffset", "homepage", "governmentType_label",
          "isPartOf_label", "areaCode", "populationTotal", "elevation",
          "maximumElevation", "minimumElevation", "populationDensity",
          "wgs84_pos#lat", "wgs84_pos#long", "areaLand", "areaMetro", "areaUrban"]

def audit_file(filename, fields):
    fieldtypes = {key: set([]) for key in fields}
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        next(reader)
        next(reader)
        next(reader)
        for row in reader:
            for field in fields:  
                if (row[field] == 'NULL') or (row[field] == ''):
                    fieldtypes[field].add(type(None))
                elif "{" in row[field]:
                    fieldtypes[field].add(type(list()))
                else: 
                    try:
                        int(row[field])    
                        fieldtypes[field].add(type(int()))
                    except ValueError:
                        try:
                            float(row[field])    
                            fieldtypes[field].add(type(float()))
                        except:
                            fieldtypes[field].add(type(str()))

    return fieldtypes


def test():
    fieldtypes = audit_file(CITIES, FIELDS)

    assert fieldtypes["areaLand"] == set([type(1.1), type([]), type(None)])
    assert fieldtypes['areaMetro'] == set([type(1.1), type(None)])
    
test()


NULL
NULL
NULL
NULL
NULL
NULL
{1.01787e+08|1.019e+08}
{3.15979e+07|3.17e+07}
{5.51667e+07|5.53e+07}
{6.36e+07|6.37137e+07}
{3.78138e+07|3.79e+07}
{2.02e+07|2.02019e+07}
{2.87489e+08|2.875e+08}
{2.5355e+07|2.5356e+07}
{2.512e+08|2.51229e+08}
{8.25e+07|8.26206e+07}
3.58195e+07
1.13e+07
5.32e+07
{4.48e+06|4.48068e+06}
{1.458e+07|1.45816e+07}
{1.71198e+07|1.712e+07}
{2.07e+06|2.07199e+06}
{4.61e+06|4.61018e+06}
{3.26e+06|3.26339e+06}
{9.057e+07|9.05719e+07}
{1.274e+07|1.27427e+07}
{3.136e+07|3.13648e+07}
{5.31466e+07|5.315e+07}
{3.43173e+07|3.432e+07}
{5.33538e+06|5.34e+06}
{8.184e+07|8.18436e+07}
{1.13959e+07|1.14e+07}
{2.056e+07|2.05645e+07}
{9.782e+07|9.78239e+07}
NULL
NULL
{4.94688e+06|4.95e+06}
{1.20175e+07|1.202e+07}
